# Routing

## Logical Routing

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import Literal
from pydantic import Field, BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


class RouteQuery(BaseModel):
    """Route the user to the most relevant datasource."""
    
    datasource : Literal["python_docs", "js_docs", "golang_docs"] = Field(..., description="Given the user question choose which of the datasource will be the most relevant.")
    
# LLM
llm = ChatOpenAI(temperature=0)

# LLM with structured output parser
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """
You are an expert in routing user's question to appropriate datasource.
Based on the programming language the question is referring to, route it to relevant datasource.  
"""

prompt = ChatPromptTemplate.from_messages(
   [
       ("system", system),
       ("human", "{question}")
   ]
)

# Router Chain
router = prompt | structured_llm  

E:\Learning-Curve\Langchain\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:1643: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [3]:
question= """
Why this code is not working?

from langchina_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
parser = StrOutputParser()

template = '''You are an helpful assistant answer the question: '''
prompt = ChatPromptTemplate.from_template(
    template
)

chain =  prompt | llm | parser
"""

result = router.invoke({"question": question})

In [4]:
# Setting up route logic

def choose_route(result):
    if "python_docs" in result.datasource.lower():
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        return "chain on js_docs"
    elif "golang_docs" in result.datasource.lower():
        return "chain on golang_docs"
    
from langchain_core.runnables import RunnableLambda

final_chain = router | RunnableLambda(choose_route)

In [5]:
result = final_chain.invoke({"question": question})

In [6]:
print(result)

chain for python_docs


## Semantic Routing

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.utils.math import cosine_similarity
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Two prompts
physics_template = """You are a very smart physics professor \n
You are great at answering the physics questions in clear and concise manner. \n
When you don't know the answer to the question, you admit it.

Here is the question: {question} 
"""

maths_template = """You are a very smart mathematics professor \n
You are great at answering the mathematics questions in clear and concise manner. \n
When you don't know the answer to the question, you admit it.

Here is the question: {question} 
"""

# Embedded Prompts

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, maths_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to the prompt
def prompt_router(input):
    # Embed input
    query_embedding = embeddings.embed_query(input['question'])
    # Computing similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]

    # Chosen prompt
    print("Using MATH" if most_similar == maths_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

chain = (
    {"question" : RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("What is blackhole?"))


Using PHYSICS
A black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it. This phenomenon occurs when a massive star collapses in upon itself, creating a dense core with a gravitational pull strong enough to trap everything within its event horizon. This means that once an object crosses the event horizon of a black hole, it is impossible for it to escape, making black holes one of the most mysterious and fascinating objects in the universe.


In [9]:
print(chain.invoke("What is Inferential statistics?"))

Using MATH
Inferential statistics is the branch of statistics that involves making inferences or conclusions about a population based on data collected from a sample of that population. It allows us to generalize our findings from the sample to the larger population. This is done through hypothesis testing, confidence intervals, and other methods to determine the likelihood of an event occurring or the relationship between variables in the population.
